In [1]:
!pip install pandas

In [2]:
import pandas as pd

In [3]:
all_nodes = pd.read_csv("all_nodes.tsv")
all_nodes.head(10)

,Node
0,GTEXEXP:ENSG00000127083-7-Esophagus-Mucosa
1,GTEXEXP:ENSG00000177542-10-Kidney-Medulla
2,PUBCHEM_CID:44419702
3,GTEXEXP:ENSG00000159023-19-Ovary
4,GTEXEXP:ENSG00000165417-11-Small-Intestine-Ter...
5,GTEXEXP:ENSG00000140522-11-Kidney-Medulla
6,GTEXEXP:ENSG00000207156-1-Vagina
7,PUBCHEM_CID:23665770
8,PUBCHEM_CID:118709697
9,GTEXEXP:ENSG00000225855-6-Fallopian-Tube


In [4]:
def get_unique_dccs(df):
    unique_dccs = (df["Node"].str.split(':').str[0]).unique()

    return unique_dccs

In [5]:
unique_dccs = get_unique_dccs(all_nodes)

In [6]:
# Creating the dictionary for nodes
nodes = {}

code_sui = pd.read_csv('neo4j/import/CODE-SUIs.csv')
code_sui.head(10)

# code_sui = code_sui.head(100)

,:END_ID,:START_ID,:TYPE,CUI
0,4-aminobenzoic acid,RXNORM:74,IN,C0000473
1,4-aminobenzoic acid,DRUGBANK:DB02362,SY,C0000473
2,4-aminobenzoic acid,CHV:0000000511,PT,C0000473
3,4-aminobenzoic acid,SNOMEDCT_US:373489001,SY,C0000473
4,4-aminobenzoic acid,MTH:NOCODE,PN,C0000473
5,Fampridine (substance),SNOMEDCT_US:80994002,FN,C0000477
6,Icosapent,SNOMEDCT_US:226367006,SY,C0000545
7,Icosapent,NCI:C68434,PT,C0000545
8,Icosapent,MSH:D015118,ET,C0000545
9,Icosapent,MTHSPL:AAN7QOV9EA,SU,C0000545


In [114]:
cuis = {}

def create_tuple_keys(row):
    sab = row[':START_ID'].split(':')[0]
    out = None
        
    if sab in unique_dccs:
        
        out = (row['CUI'], sab, row[':TYPE'])

        if out not in cuis:
            cuis[out] = {
                "label": row[':END_ID'],
                "label_synonyms": []
            }

        cuis[out]["label_synonyms"].append(row[':END_ID'])
    
    return out

In [74]:
# code_sui['CUI_START_ID_KEY'] = code_sui.apply(lambda row: f"({row['CUI']}, {row[':START_ID'].split(':')[0]}, {row[':TYPE']})", axis=1)

In [115]:
code_sui['CUI_START_ID_KEY'] = code_sui.apply(create_tuple_keys, axis=1)
print(code_sui.shape, len(code_sui['CUI_START_ID_KEY'].unique()))
code_sui = code_sui[code_sui['CUI_START_ID_KEY'].notna()]

(1362767, 5) 1148638


In [116]:
print(len(cuis))

1148638


In [112]:
print(code_sui.head(10), code_sui.shape)

                            :END_ID              :START_ID :TYPE       CUI  \
3               4-aminobenzoic acid  SNOMEDCT_US:373489001    SY  C0000473   
5            Fampridine (substance)   SNOMEDCT_US:80994002    FN  C0000477   
6                         Icosapent  SNOMEDCT_US:226367006    SY  C0000545   
14   Neoplasm of abdomen (disorder)  SNOMEDCT_US:128050000    FN  C0000735   
16                     Birth defect  SNOMEDCT_US:276720006    SY  C1737329   
17  Incomplete spontaneous abortion  SNOMEDCT_US:156072005    SY  C0000810   
20                 Academic problem    SNOMEDCT_US:3677008    PT  C0000873   
24                              Cut  SNOMEDCT_US:283396008    SY  C0000925   
26                              Cut  SNOMEDCT_US:134352000    SY  C0000925   
42                  Achondrogenesis    SNOMEDCT_US:2391001    PT  C0001079   

               CUI_START_ID_KEY  
3   (C0000473, SNOMEDCT_US, SY)  
5   (C0000477, SNOMEDCT_US, FN)  
6   (C0000545, SNOMEDCT_US, SY)  
14  (

In [77]:
codes = pd.read_csv('neo4j/import/CODEs.csv', dtype = {'CODE': str})
codes = codes[codes['SAB'].isin(unique_dccs)]
codes.head(10)

,CodeID:ID,SAB,CODE,value:float,lowerbound:float,upperbound:float,unit
6,SNOMEDCT_US:28677006,SNOMEDCT_US,28677006,NaN,NaN,NaN,NaN
32,SNOMEDCT_US:14423008,SNOMEDCT_US,14423008,NaN,NaN,NaN,NaN
33,SNOMEDCT_US:55603005,SNOMEDCT_US,55603005,NaN,NaN,NaN,NaN
44,SNOMEDCT_US:15890002,SNOMEDCT_US,15890002,NaN,NaN,NaN,NaN
56,SNOMEDCT_US:714291009,SNOMEDCT_US,714291009,NaN,NaN,NaN,NaN
78,SNOMEDCT_US:53800008,SNOMEDCT_US,53800008,NaN,NaN,NaN,NaN
91,SNOMEDCT_US:1217427007,SNOMEDCT_US,1217427007,NaN,NaN,NaN,NaN
121,SNOMEDCT_US:96149000,SNOMEDCT_US,96149000,NaN,NaN,NaN,NaN
139,SNOMEDCT_US:117497006,SNOMEDCT_US,117497006,NaN,NaN,NaN,NaN
140,SNOMEDCT_US:13746004,SNOMEDCT_US,13746004,NaN,NaN,NaN,NaN


In [78]:
print(code_sui.shape, codes.shape, len(code_sui['CUI_START_ID_KEY'].unique()))

(1362767, 5) (445586, 7) 1148638


In [79]:
print(len(codes['CodeID:ID'].unique()), len(code_sui[':START_ID'].unique()))

445586 445586


In [82]:
maps = {}

maps = codes.set_index("CodeID:ID").T.to_dict('list')

In [83]:
unique_code_sui = code_sui.drop_duplicates(subset = [':START_ID'], inplace = False)
duplicate_code_sui = code_sui[code_sui.duplicated([':START_ID'], keep='first')]
print(unique_code_sui.shape,duplicate_code_sui.shape)

(445586, 5) (917181, 5)


In [84]:
merged =  unique_code_sui.merge(codes, left_on = ':START_ID', right_on = 'CodeID:ID').drop(['CodeID:ID'], axis = 1)
print(merged.shape, merged.head(10))

(445586, 11)                            :END_ID              :START_ID :TYPE       CUI  \
0              4-aminobenzoic acid  SNOMEDCT_US:373489001    SY  C0000473   
1           Fampridine (substance)   SNOMEDCT_US:80994002    FN  C0000477   
2                        Icosapent  SNOMEDCT_US:226367006    SY  C0000545   
3   Neoplasm of abdomen (disorder)  SNOMEDCT_US:128050000    FN  C0000735   
4                     Birth defect  SNOMEDCT_US:276720006    SY  C1737329   
5  Incomplete spontaneous abortion  SNOMEDCT_US:156072005    SY  C0000810   
6                 Academic problem    SNOMEDCT_US:3677008    PT  C0000873   
7                              Cut  SNOMEDCT_US:283396008    SY  C0000925   
8                              Cut  SNOMEDCT_US:134352000    SY  C0000925   
9                  Achondrogenesis    SNOMEDCT_US:2391001    PT  C0001079   

              CUI_START_ID_KEY          SAB       CODE  value:float  \
0  (C0000473, SNOMEDCT_US, SY)  SNOMEDCT_US  373489001          NaN 

In [104]:
# idx = 0
i = {"0":0}
def get_values_from_merged(row):
    # idx += 1
    # if idx % 10 == 0:
    #     print(idx, row.name)
    if i["0"] % 10 == 0:
        print(i)
    i["0"] += 1
    value = maps[row[':START_ID']]
    # print(value)
    # if row[':START_ID'] not in values:
    #     res = codes.loc[codes['CodeID:ID'] == row[':START_ID']]
    #     values[row[':START_ID']] = res[['SAB','CODE','value:float','lowerbound:float','upperbound:float','unit']]
    #     print("inside", len(values))
        
    return value

In [105]:
new_df = pd.DataFrame()

new_df = duplicate_code_sui.copy()

new_df = new_df.reindex(columns= merged.columns)
print(new_df.shape)

# new_df['SAB','CODE','value:float','lowerbound:float','upperbound:float','unit'] = codes['SAB','CODE','value:float','lowerbound:float','upperbound:float','unit'].where()


(917181, 11)


In [106]:
new_df[['SAB','CODE','value:float','lowerbound:float','upperbound:float','unit']] = new_df.apply(get_values_from_merged,axis= 1, result_type= 'expand'

{'0': 0}
{'0': 10}
{'0': 20}
{'0': 30}
{'0': 40}
{'0': 50}
{'0': 60}
{'0': 70}
{'0': 80}
{'0': 90}
{'0': 100}
{'0': 110}
{'0': 120}
{'0': 130}
{'0': 140}
{'0': 150}
{'0': 160}
{'0': 170}
{'0': 180}
{'0': 190}
{'0': 200}
{'0': 210}
{'0': 220}
{'0': 230}
{'0': 240}
{'0': 250}
{'0': 260}
{'0': 270}
{'0': 280}
{'0': 290}
{'0': 300}
{'0': 310}
{'0': 320}
{'0': 330}
{'0': 340}
{'0': 350}
{'0': 360}
{'0': 370}
{'0': 380}
{'0': 390}
{'0': 400}
{'0': 410}
{'0': 420}
{'0': 430}
{'0': 440}
{'0': 450}
{'0': 460}
{'0': 470}
{'0': 480}
{'0': 490}
{'0': 500}
{'0': 510}
{'0': 520}
{'0': 530}
{'0': 540}
{'0': 550}
{'0': 560}
{'0': 570}
{'0': 580}
{'0': 590}
{'0': 600}
{'0': 610}
{'0': 620}
{'0': 630}
{'0': 640}
{'0': 650}
{'0': 660}
{'0': 670}
{'0': 680}
{'0': 690}
{'0': 700}
{'0': 710}
{'0': 720}
{'0': 730}
{'0': 740}
{'0': 750}
{'0': 760}
{'0': 770}
{'0': 780}
{'0': 790}
{'0': 800}
{'0': 810}
{'0': 820}
{'0': 830}
{'0': 840}
{'0': 850}
{'0': 860}
{'0': 870}
{'0': 880}
{'0': 890}
{'0': 900}
{'0': 910}

In [138]:
def get_labels(row):
    # print(len(cuis[row["CUI_START_ID_KEY"]]['label_synonyms']))
    return cuis[row["CUI_START_ID_KEY"]]['label_synonyms']

In [139]:
all_merged_df = pd.concat([merged, new_df])
final_df = all_merged_df[["CUI", "SAB", "CODE", ":END_ID", "CUI_START_ID_KEY", "value:float", "lowerbound:float", "upperbound:float", "unit"]]

col_map = {
    "CUI": "ID",
    ":END_ID": "label",
    "value:float":"value",
    "lowerbound:float": "lowerbound",
    "upperbound:float": "upperbound"
}
final_df = final_df.rename(columns = col_map)
print(final_df.columns)
final_df["label_synonyms"] = final_df.apply(get_labels, axis = 1)

Index(['ID', 'SAB', 'CODE', 'label', 'CUI_START_ID_KEY', 'value', 'lowerbound',
       'upperbound', 'unit'],
      dtype='object')


In [142]:
final_df.head(50)
final_df.to_csv('nodes.tsv', sep="\t")